In [ ]:
from imageio import imread
import os
import numpy as np
import pandas as pd

In [ ]:
def load_and_flatten_image(imagePath):
    image = imread(imagePath)
    image_single_channel = image[:,:,0]
    image_array = image_single_channel[-1]
    return image_array

In [ ]:
path = "classificatie"
cpt = sum([len(files) for r, d, files in os.walk(path)])
data_array = np.empty([cpt, 352*640], dtype=np.uint8)
label_array = np.empty([cpt], dtype=np.str_)
counter = 0
for root, dirs, files in os.walk(path):
    for dirname in dirs:
        for root2, dirs2, files2 in os.walk(os.path.join(path, dirname)):
            for filename in files2:
                image = load_and_flatten_image(os.path.join(os.path.join(path, dirname), filename))
                for key, pixel in enumerate(image):
                    data_array[counter][key] = pixel
                label_array[counter] = dirname
                counter += 1

dataset = pd.DataFrame({'label': label_array, 'images': list(data_array)}, columns=['label', 'images'])
dataset.head()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tel = 0
for train_index, test_index in split.split(dataset, dataset["label"]):
    tel += 1
    print(tel)
    strat_train_set = dataset.loc[train_index]
    strat_test_set = dataset.loc[test_index]

In [ ]:
X_train, y_train = strat_train_set["images"], strat_train_set["label"]

In [ ]:
y_train_n = (y_train == 'n')

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(X_train, y_train_n)

In [ ]:
X_test, y_test = strat_test_set["images"], strat_test_set["label"]
some_image = X_test[0]
sgd_clf.predict([some_image])